In [ ]:
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv')

In [ ]:
movies.head(2)

In [ ]:
credits.head(2)

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies = movies.merge(credits,on = 'title')

In [ ]:
movies.head(2)

In [ ]:
movies.shape

In [ ]:
movies.columns

In [ ]:
movies = movies[['movie_id', 'title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.shape

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['genres']

In [ ]:
import ast

def convert(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])

    return l

In [ ]:
movies['genres']= movies['genres'].apply(convert)

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['keywords']

In [ ]:
movies['keywords']= movies['keywords'].apply(convert)
movies.head(2)

In [ ]:
movies.iloc[0]['cast']

In [ ]:
import ast

def convert_cast(text):
    l = []
    counter = 0

    # Only parse if it's a string
    if isinstance(text, str):
        try:
            text = ast.literal_eval(text)
        except:
            return []

    for i in text:
        if counter < 3:
            # If it's already a list of strings, return it directly
            if isinstance(i, str):
                l.append(i)
            elif isinstance(i, dict) and 'name' in i:
                l.append(i['name'])
            counter += 1

    return l


In [ ]:
movies['cast']= movies['cast'].apply(convert_cast)
movies.head(2)

In [ ]:
movies.iloc[0]['crew']

In [ ]:
def fetch_director(text):
    l=[]
    for i in ast.literal_eval(text):
        if i['job'] =='Director':
            l.append(i['name'])
            break

    return l

In [ ]:
movies['crew']= movies['crew'].apply(fetch_director)
movies.head(2)

In [ ]:
movies.iloc[0]['overview']

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies.head(2)

In [ ]:
movies.iloc[0]['overview']

In [ ]:
movies.head()

In [ ]:
#Sam Worthington
#SamWorthington

def remove_space(word):
    l= []
    for i in word:
        l.append(i.replace(" ",""))

    return l    

In [ ]:
movies['cast'] = movies['cast'].apply(remove_space)
movies['crew'] = movies['crew'].apply(remove_space)
movies['genres'] = movies['genres'].apply(remove_space)
movies['keywords'] = movies['keywords'].apply(remove_space)

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
movies.iloc[0]['overview']

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']]


In [ ]:
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
new_df.shape

In [ ]:
import nltk
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    l =[]
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stems)

In [ ]:
new_df.iloc[0]['tags']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vector

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similary = cosine_similarity(vector)

In [ ]:
similary

In [ ]:
similary.shape

In [ ]:
new_df[new_df['title'] == 'Spider-Man'].index[0]

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similary[index])), reverse = True , key = lambda x:x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Spider-Man')

In [ ]:
recommend('The Dark Knight Rises')

In [ ]:
import pickle

pickle.dump(new_df, open('artifacts/movie_list.pkl', 'wb'))
pickle.dump(similary, open('artifacts/similarity.pkl', 'wb'))

In [ ]:
recommend('The Purge')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(), open('movie_dict.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
new_df.to_dict()

In [ ]:
pickle.dump(similary,open('similary.pkl', 'wb'))